In [12]:
import csv
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as plcolor

In [13]:
def my_layout(G, scale=100):
    layout = {}
    for node in G.nodes():
        layout[node] = [G.nodes[node]['x'] / scale, G.nodes[node]['y'] / scale]
    return layout

In [14]:
cdict = {
    'red'   : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'green' : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'blue'  : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'alpha' : [[0.0, 0.0, 0.0],
               [1.0, 1.0, 1.0]]
}
black_alpha = plcolor.LinearSegmentedColormap('black_alpha', cdict)

In [56]:
def total_degree(G, weight_str):
    sum_degree = 0
    for player, d in G.out_degree(weight=weight_str):
        sum_degree += d
    return sum_degree

In [81]:
def avg_position(G):
    sum_x = 0.0
    sum_y = 0.0
    num_node = G.number_of_nodes()
    for n in G.nodes():
        if 'x' in G.nodes[n].keys():
            sum_x += G.nodes[n]['x']
            sum_y += G.nodes[n]['y']
        else:
            num_node -= 1
    avg_pos = (sum_x / num_node, sum_y / num_node)
    return avg_pos

In [82]:
def global_describe(G, weight_str):
    avg_clu = nx.average_clustering(G, weight=weight_str)
    tot_deg = total_degree(G, weight_str)
    max_cli = nx.graph_clique_number(G.to_undirected())
    avg_pos = avg_position(G)
    dict_des = {'avg_clustering': avg_clu, 'total_degree': tot_deg, 'clique_number': max_cli, 'avg_position': avg_pos}
    return dict_des

In [83]:
def gen_cal_graph(team_name):
    G = nx.DiGraph()
    with open('2020_Problem_D_DATA/weight.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if team_name in row['OriginID'] and team_name in row['DestinationID']:
                if G.has_edge(row['OriginID'], row['DestinationID']):
                    G[row['OriginID']][row['DestinationID']]['weight'] += int(row['Weight'])
                else:
                    G.add_edge(row['OriginID'], row['DestinationID'], weight=int(row['Weight']))

    with open('2020_Problem_D_DATA/macro_location.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if team_name in row['OriginPlayerID']:
                G.nodes[row['OriginPlayerID']]['x'] = float(row['EventOrigin_x'])
                G.nodes[row['OriginPlayerID']]['y'] = float(row['EventOrigin_y'])
    total_dict = global_describe(G, 'weight')
    total_dict['teamID'] = team_name
    return total_dict

In [84]:
team_names = ['Huskies','Opponent1','Opponent2','Opponent3','Opponent4','Opponent5','Opponent6','Opponent7','Opponent8','Opponent9','Opponent10','Opponent11','Opponent12','Opponent13','Opponent14','Opponent15','Opponent16','Opponent17','Opponent18','Opponent19']
total_list = []
for name in team_names:
    total_list.append(gen_cal_graph(name))

In [87]:
headers = ['teamID', 'avg_clustering', 'total_degree', 'clique_number', 'avg_position']
with open('global_indicator.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_list)